# [基础torchtext数据预处理] 

In [1]:
import torch 
import torchtext
from torchtext import data,datasets
#import spacy  也可以换成spacy分词工具，242上安装了spacy，246上安装了pyltp.
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
list_stopwords=list(set(stopwords.words('english')))

print(torch.__version__)
print(torchtext.__version__)


1.3.1
0.4.0


In [2]:
def splitsentence(text):
    sentences=sent_tokenize(text)
    return sentences
def splitword(sentence):
    words=word_tokenize(sentence)
    return words

In [3]:
if __name__=='__main__':
    print(splitsentence("My name is tom. He has a big head. I like soccer!"))
    print(splitword("My name is tom."))

['My name is tom.', 'He has a big head.', 'I like soccer!']
['My', 'name', 'is', 'tom', '.']


In [4]:
TEXT=data.Field(sequential=True,tokenize=splitword,lower=True,fix_length=150,stop_words=list_stopwords,batch_first=True)
# LABEL=data.Field(sequential=False,use_vocab=False)  #官方文档的写法
LABEL=data.LabelField(dtype=torch.long)

train,val,test=data.TabularDataset.splits(path='./ose_data_2/',
                                         train='train_2.tsv',
                                         validation='validation_2.tsv',
                                         test='test_2.tsv',
                                         format='tsv',
                                         skip_header=True,
                                         fields=[('Label',LABEL),('Text',TEXT)])
#TEXT.build_vocab(train)
#from torchtext.vocab import Vectors
#vectors = Vectors(name='/home/yuling/tyling-data/word_vectors/glove.6B.100d.txt')
# 指定 Vector 缺失值的初始化方式，没有命中的token的初始化方式
from torchtext.vocab import GloVe
TEXT.build_vocab(train,val,vectors='glove.6B.100d')
LABEL.build_vocab(train,val)
# 查看词表元素
TEXT.vocab.vectors
vocab=TEXT.vocab
vocab
#TEXT.vocab.freqs.most_common(10)

In [5]:
# for i in range(len(train)):
#     print(train[i].Label)
# for i in range(len(val)):
#     print(val[i].Label)

In [6]:
train_iter,val_iter,test_iter=data.Iterator.splits((train,val,test),
                                                   sort_key=lambda x:len(x.Text),
                                                   batch_sizes=(32,256,256))
                                                  #device=-1)print(train[5])
 #查看数据
for batch_idx,(X_train_var,y_train_var) in enumerate(train_iter):
    print(X_train_var.shape,X_train_var)
    print(y_train_var.shape, y_train_var)
    break

torch.Size([32, 150]) tensor([[  851,   123,   184,  ...,   738,    30,  1603],
        [  141,     2,  1164,  ...,  5516,  2748,     3],
        [  102,  4144,   162,  ...,   433,   427,   522],
        ...,
        [  125,   888,   815,  ...,    25,   385,   527],
        [  312,    55,  1696,  ...,     4,     3,   212],
        [16976,  1612,  3668,  ...,     2,   154,     3]])
torch.Size([32]) tensor([0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 1, 2, 0, 1, 0, 1, 2, 2, 0, 2,
        2, 1, 2, 0, 2, 2, 2, 1])


In [7]:
# print(train[5])
# print(train[5].__dict__.keys())

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# [网络结构]

1. 与维度变换相关的函数 view() , permute() , torch.squeeze() , torch.unsqueeze()     
2. Embedding层加载预训练模型的方式： 1) copy , 2) from_pretrained    
3. LSTM 参数以及输入输出说明： 结构参数 LSTM(input_size,hidden_size,num_layers)   
   input_size:输入的特征数量    
   hidden_size:隐藏的特征数量     
   num_layers:层数  
4. Linear 的创建需要两个参数，inputsize 和 outputsize    
   inputsize:输入节点数    
   outputsize：输出节点数    
   所以Linear有7个字段：    
       weight: Tensor , outputsize x inputsize    
       bias: Tensor , outputsize    
       gradWeight: Tensor , outputsize x inputsize    
       gradbias: Tensor , outputsize    
       gradinput: Tensor    
       output: Tensor   
       _type: output:type()

In [8]:
weight_matrix=TEXT.vocab.vectors
len_vocab = len(TEXT.vocab)

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Enet(nn.Module):
    def __init__(self):
        super(Enet,self).__init__()
        
        self.embedding=nn.Embedding(len_vocab,100)
        
        #若使用预训练的词向量，需要指定预训练的权重;
        #此处的pretrained_embeddings即为权重矩阵 weight_matrix;
        #self.embedding = nn.Embedding.from_pretained(pretrained_embeddings,freeze=False)
        
        self.lstm = nn.LSTM(100,64,3,batch_first=True)#,bidirectional=True)
        self.linear = nn.Linear(64,3)
    
    def forward(self,x):
        text = x
        batch_size,seq_num = text.shape #[32,150]
        vec = self.embedding(text)  #[32,150,100]
        out,(hn,cn) = self.lstm(vec)  #[32,150,64]
        out = self.linear(out[:,-1,:])  #[32,3]
        out = F.softmax(out,-1)
        
        #print("x.shape:",x.shape) [32,150]    
        #32即为batch的大小，150为batch中每个样本的序列长度       
        #print("vec.shape:",vec.shape) [32,150,100]
        #100即为序列中每个词向量的维度
        #print("out.shape:",out.shape) [32,150,64]
        #因为设定的LSTM中词向量的维度是64，所以经过LSTM之后维度从100变为64
        #print("linear_out.shape:",out.shape) [32,3]
        #经过全连接层之后降维
        #经过softmax之后可以得出在每个分支上的概率
        
        
        return out

In [10]:
# 模型单独测试
net = Enet()
# net.to(device)
for i in train_iter:
    print(i.Text)
    print(i.Text.shape)
    net.forward(i.Text)
    break
#net.train()

tensor([[    7,    58,    78,  ...,    46,     2,  1260],
        [ 2908,  2953,  2388,  ...,  6979,  1297,     2],
        [19404,  1145,     2,  ..., 18367,     2,  5056],
        ...,
        [  233,  4142,  1664,  ...,    68,  6413,     3],
        [ 8307,  2135,   329,  ...,   695,  2464,   354],
        [   15,    13,    55,  ...,  4938,  1415,   879]])
torch.Size([32, 150])


# [训练]

In [11]:
optimizer = optim.Adam(filter(lambda p:p.requires_grad, net.parameters()),lr=0.01)
loss_function = F.cross_entropy   #用交叉熵作为loss优化的评判标准

In [12]:
for batch_idx, batch in enumerate(train_iter):
    print(batch_idx)   
    optimizer.zero_grad()
    predicted = net(batch.Text)
    #print(predicted)
    #print(batch.Label)
    loss = loss_function(predicted, batch.Label)
    loss.backward()
    optimizer.step()
    print(loss)
    break

0
tensor(1.0971, grad_fn=<NllLossBackward>)


# [TextRNN 模型] 

In [26]:
class TextRNN(nn.Module):
    def __init__(self, embedding_dim, output_dim, hidden_size, num_layers, bidirectional, pretrained_embeddings):

        super(TextRNN, self).__init__()
    
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.output_dim = output_dim   #output_dim可以表示输出维度，也可表示标签个数
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        #self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        self.lstm = nn.LSTM(self.embedding_dim,
                            self.hidden_size, 
                            self.num_layers, 
                            batch_first=True, 
                            #第一个维度设为batch，即（batch_size，seq_length，embedding_dim）
                            bidirectional=self.bidirectional)
        self.fc = nn.Linear(self.hidden_size*2, output_dim)  #本例中一律用双向
        #self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        text = x
        batch_size , seq_length = text.shape
        #print("batch_size:",batch_size)
        #vec = self.dropout(self.embedding(text))
        #vec = vec.permute(1,0,2)
        vec = self.embedding(text)
        #print('\nvec.shape:',vec.shape)
        #以下为初始化h0 和 c0
        h0 = torch.zeros(self.num_layers*2,batch_size,self.hidden_size)
        c0 = torch.zeros(self.num_layers*2,batch_size,self.hidden_size)
        
        lstm_out, (hn,cn) = self.lstm(vec,(h0,c0))
        
        #print('\nlstm_out.shape:',lstm_out.shape)
        #我们只需要最后一步的输出，即（batch_size,-1,output_size）
        out = self.fc(lstm_out[:, -1, :])
        #print('fc_out.shape:',out.shape)
        return out
        


In [28]:
pretrained_embeddings = TEXT.vocab.vectors
embedding_dim = 100
output_dim = 3
hidden_size = 64
num_layers = 5
bidirectional = True
#dropout = 0.5

model = TextRNN(embedding_dim, output_dim, hidden_size, num_layers, bidirectional, pretrained_embeddings)

for i in train_iter:
    model.forward(i.Text)

# [导入TextCNN模型进行训练]

In [32]:
from TextCNN import TextCNN

#测试模型
pretrained_embeddings = TEXT.vocab.vectors
embedding_dim =  100
output_dim = 3
n_filters = 64  #卷积核的个数
filter_sizes = [2,3,4]
dropout = 0.7

model = TextCNN(embedding_dim, n_filters, filter_sizes, output_dim, pretrained_embeddings, dropout)
for i in train_iter:
    print(i.Text,i.Text.shape)
    model.forward(i.Text)

tensor([[ 332,  678, 1358,  ...,  670,  853, 1359],
        [1550,  481, 4737,  ..., 3795, 8195, 2942],
        [1185,  804, 1084,  ..., 2574, 1139,  445],
        ...,
        [3980, 2336,  129,  ...,  480, 6923, 2030],
        [7752, 2315, 4795,  ..., 1104,    6,  313],
        [8821,  807, 8578,  ...,  113, 4917,   83]]) torch.Size([32, 150])
tensor([[  233,   437,  1664,  ...,   716,   271,  2322],
        [   33,  1672,     2,  ..., 20198,  1452,     2],
        [  119,     2,   510,  ...,  2936,  8117,  2353],
        ...,
        [ 1052, 15096,   123,  ...,     2,  8081,  1480],
        [ 4172,  1356,     2,  ...,  3822,  1725,  2527],
        [ 2281,    84,   153,  ...,     2,  6072,  4560]]) torch.Size([32, 150])
tensor([[  141,  6133,   545,  ...,  5395,     3,  1035],
        [ 1942,   391,  2908,  ...,  1078,   231,    70],
        [   62,   882,     2,  ..., 19451,   156,  2680],
        ...,
        [ 1741,  2473,   360,  ...,   201,     6,   552],
        [   36,   413, 

# [导入TextRCNN模型进行训练]

In [49]:
from TextCNN import TextRCNN

#测试模型
pretrained_embeddings = TEXT.vocab.vectors
embedding_dim = 100
output_dim = 3
hidden_size = 64
num_layers = 5
dropout = 0.5

model = TextRCNN(embedding_dim, output_dim, hidden_size,num_layers, pretrained_embeddings, dropout)
for i in train_iter:
    print(i.Text, i.Text.shape)
    model.forward(i.Text)

AttributeError: 'float' object has no attribute 'dim'